In [1]:
!pip freeze | grep scikit-learn


scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1598376899566/work


In [2]:
!python -V


Python 3.8.5


In [3]:
import pickle
import pandas as pd

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ibai/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/ibai/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [7]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')



In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [14]:
#Q1:
df_results = pd.DataFrame()
df_results["predicted_duration"] = y_pred
print(df_results.std())


predicted_duration    6.24749
dtype: float64


In [16]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
df_results['ride_id'] = df['ride_id']

In [20]:
df_results.to_parquet(
    f'pred_taxi_{year}_{month}',
    engine='pyarrow',
    compression=None,
    index=False
)

In [21]:
df_results

# Q2: 66Mb.

,predicted_duration,ride_id
0,16.245906,2023/03_0
1,26.134796,2023/03_1
2,11.884264,2023/03_2
3,11.997720,2023/03_3
4,10.234486,2023/03_4
...,...,...
3316211,11.952635,2023/03_3316211
3316212,20.049958,2023/03_3316212
3316213,11.595336,2023/03_3316213
3316214,13.113178,2023/03_3316214


In [22]:
#Q3: jupyter nbconvert --to script starter.ipynb

In [27]:
import sklearn

In [28]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))


pandas==1.1.3
pip==20.2.4
sklearn==0.23.2


In [30]:
# pipenv install scikit-learn==0.23.2 --python=3.8.5